## Assignment 1: Build a Predictor
## Dhruval Bhatt (Collaborated with Keertana Chidambaram)

Instructions from assignment:

Build a predictor . Once you have built a predictor, you should use the test data set to submit a
csv file on canvas. That csv file should only have two columns: diag_id, y_hat.

y_hat = drinks per day = 'U1031900'
diag_id = 'diag.id'

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [2]:
# Load the data given

train_df = pd.read_csv('../Data/nlsy training set.csv')
test_df = pd.read_csv('../Data/nlsy test set.csv')

print("Size of Training Set", train_df.shape)
print("Size of Test Set", test_df.shape)

train_df.drop(train_df.columns[[0]], axis=1, inplace=True)  #remove unnamed column that is the repeat of id column in train
test_df.drop(test_df.columns[[0]], axis=1, inplace=True)#remove unnamed column that is the repeat of id column in test

print("Size of Training Set after Removing Col 1", train_df.shape)
print("Size of Test Set after Removing Col 1", test_df.shape)

Size of Training Set (7187, 4887)
Size of Test Set (1797, 4886)
Size of Training Set after Removing Col 1 (7187, 4886)
Size of Test Set after Removing Col 1 (1797, 4885)


In [3]:
train_df.set_index('diag.id', inplace=True)

### Data Prep

In [4]:
from sklearn.model_selection import train_test_split

In [7]:
# Remove rows that have missing values for the target data

train_sub = train_df[train_df['U1031900'] >= 0] #subset 

In [ ]:
train_X = train_df.loc[:, train_df.columns != 'U1031900']
print("Size of Training Set X variables", train_X.shape)

In [ ]:
train_y = train_df['U1031900']
print("Size of Training Set y variables", train_y.shape)

In [ ]:
# for col in train_X.columns:
#     missing_num = col.where(df < 0).count()
#     tot = len(col)
#     if (missing_num/tot*100 < 70):
#         mode = col.mode()
#         col[col < 0 ] = mode 
        
#     new_cols = list(new_dummies.columns)
#     new_cols = [str(i) + col for i in new_cols]
#     new_dummies.columns = new_cols
#     train_X = train_X.join(new_dummies)

In [ ]:
# for col in train_X.columns:
#     new_dummies = pd.get_dummies(train_X[col])
#     new_cols = list(new_dummies.columns)
#     new_cols = [str(i) + col for i in new_cols]
#     new_dummies.columns = new_cols
#     train_X = train_X.join(new_dummies)

# train_X.head()

In [ ]:
# Split into train and test 
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.2, random_state=0)

In [ ]:
X_train.shape

### Feature Selection 

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import make_scorer
import numpy as np

In [ ]:
# Decision Tree

model_1 = tree.DecisionTreeClassifier()
model_1.fit(X_train, y_train)
pred_1 = model_1.predict(X_test)
mse_1 = mean_squared_error(y_test, pred_1)
print('MSE: %f' % mse_1)

In [ ]:
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index= X_train.columns)
#feat_importances.nlargest(10).plot(kind='barh')
#plt.show()

selected_features_1 = list(feat_importances.nlargest(2000).index)

In [ ]:
# Decision Tree with Selected (Just kept the top 2000 predictors)

X_train_sel = X_train.filter(selected_features_1)
X_test_sel = X_test.filter(selected_features_1)

model_2 = tree.DecisionTreeClassifier()
model_2.fit(X_train_sel, y_train)
pred_2 = model_2.predict(X_test_sel)
mse_2 = mean_squared_error(y_test, pred_2)
print('MSE: %f' % mse_2)

In [ ]:
# Random Forest Decision Tree

model_3 = RandomForestClassifier()
model_3.fit(X_train, y_train)
pred_3 = model_3.predict(X_test)
mse_3 = mean_squared_error(y_test, pred_3)
print('MSE: %f' % mse_3)

In [ ]:
# Code Missing Variables

from sklearn.preprocessing import OneHotEncoder
t_X_train = X_train
t_X_train[t_X_train < 0] = 999
t_X_train.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_columns = list(t_X_train.columns)
cat_columns_idx = [t_X_train.columns.get_loc(col) 
                   for col in cat_columns]
ohe = OneHotEncoder(categorical_features=cat_columns_idx, 
                    sparse=False, handle_unknown="ignore")
df_processed_np = ohe.fit(t_X_train)

In [ ]:
#X = 
pd.get_dummies(X_train)#, prefix_sep='_', drop_first=True)
#X_train['E5011701']
#pd.get_dummies(X_train['E5011701'])#, prefix_sep='_', drop_first=True)

In [ ]:
results_df = pd.DataFrame({"model": ["Decision Tree", "Decision Tree Selected", "Decision Tree Tuned", "Random Forest", "Random Forest Tuned"],
                   "MSE": [mse_1, mse_2, "NA for Now", mse_3, "NA for Now"]})

In [ ]:
results_df

In [ ]:
def calc_MSE(y_true, y_pred):
    '''Calculates the Mean Squared Error given original and predicted values'''
    y_pred[y_pred<0] = 0
    err = (y_pred[y_true>=0] - y_true[y_true>=0]) ** 2
    return err.mean()
new_scorer = make_scorer(calc_MSE, greater_is_better=False)

In [ ]:
# XgBoost
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [ ]:
#XGB Regressor
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
yhat = np.around(preds)
calc_MSE(y_test, yhat)

In [ ]:
len(y_train.unique())

In [ ]:
#XGB Classifier

clf = xgb.XGBClassifier(objective='multi:softmax', n_estimators = 2, max_depth  = 2)
# train1 = clf.predict_proba(train_data)
# test1 = clf.predict_proba(test_data)


In [ ]:
clf.fit(X_train, y_train)  

In [ ]:
preds_c = clf.predict(X_test)

In [ ]:
preds_c

In [ ]:
#yhat_c = np.around(preds)
calc_MSE(y_test, preds_c)

In [ ]:
diag = pd.DataFrame(X_test.index)
predsc = pd.DataFrame(preds_c)
df = pd.concat([diag, predsc], axis=1, sort=False)

In [ ]:
test_df.head()